# Node embedding codes for the graph feature engineering

In [2]:
!pip install neo4j

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 4.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.14.1-py3-none-any.whl size=267560 sha256=f0f9453425d6c5e8fd594c48c3089accf51b4a6065edbeb253839144627e73e9
  Stored in directory: /home/sookyoung.kim/.cache/pip/wheels/1f/e5/4b/eb5810827108f5f9fcbdd43b1f2739dd2ecc65841d744777cf
Successfully built neo4j

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install netifaces

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


# 1. Module import 및 실행 환경 확인

In [29]:
import pandas as pd
import numpy as np

import neo4j
import netifaces
import os
import socket
import re

import pickle, os, datetime, time
import sqlite3

from netifaces import interfaces, ifaddresses, AF_INET
from neo4j import GraphDatabase
from neo4j import DEFAULT_DATABASE
from neo4j import WRITE_ACCESS
from neo4j import unit_of_work
from neo4j import Query

from datetime import datetime

In [30]:
this_env = "LS"  # MC or LS (model conductor or local server)

if this_env == "MC":
    # At Model conductor
    # Model conductor의 gp-spend-intelligence project의 gpsi-weekly workspace에서만 참조 가능
    # data_input.py가 Emro data를 저장하는 경로를 공유하기 위함
    from mc_asset import Asset
else:
    # At local    
    # Local machine에서 GPU를 사용할 수 있는 지 확인
    import torch
    #%tensorflow_version 2.x
    import tensorflow as tf

    print("Pytorch version = {}".format(torch.__version__))
    print("Tensorflow version = {}".format(tf.__version__))

    print("Pysical devices that Tensoflow detected = {}".format(tf.config.list_physical_devices()))
    physical_devices = tf.config.list_physical_devices('GPU')

    device_name = tf.test.gpu_device_name()
    print('device_name: {}'.format(device_name))
    if device_name != '/device:GPU:0':
      raise SystemError('GPU device not found')
    print('Found GPU at: {}'.format(device_name))

Pytorch version = 1.13.0+cu117
Tensorflow version = 2.4.0
Pysical devices that Tensoflow detected = [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]
device_name: /device:GPU:0
Found GPU at: /device:GPU:0


# 2. 실행 환경 변수 설정

In [31]:
# 현재 코드가 실행되고 있는 경로 출력
print("\nCode path -----------------------------------------------------------")
print(os.getcwd())
print("-----------------------------------------------------------------------")

# 컨테이너 내부에서 ip address 확인
print("\nnetifaces -----------------------------------------------------------")
print(netifaces.interfaces())
print(netifaces.ifaddresses('lo'))
if this_env == "MC":
    print(netifaces.ifaddresses('eth0'))
    ADDRESS = ifaddresses("eth0")[2][0]["addr"]
    print("eth0 address = ", ADDRESS)
else:
    print(netifaces.ifaddresses('eno1'))
    ADDRESS = ifaddresses("eno1")[2][0]["addr"]
    print("eno1 address = ", ADDRESS)
print("-----------------------------------------------------------------------")


Code path -----------------------------------------------------------
/ssd_data/sookyoung_ssd/projects/GraphDB/customer_index/complaint_index
-----------------------------------------------------------------------

netifaces -----------------------------------------------------------
['lo', 'eno1', 'eno2d1', 'docker0']
{17: [{'addr': '00:00:00:00:00:00', 'peer': '00:00:00:00:00:00'}], 2: [{'addr': '127.0.0.1', 'netmask': '255.0.0.0', 'peer': '127.0.0.1'}]}
{17: [{'addr': 'f4:02:70:a5:52:ca', 'broadcast': 'ff:ff:ff:ff:ff:ff'}], 2: [{'addr': '10.158.2.51', 'netmask': '255.255.255.0', 'broadcast': '10.158.2.255'}]}
eno1 address =  10.158.2.51
-----------------------------------------------------------------------


In [32]:
# 환경 변수 설정 
if this_env == "MC":
    my_mc_asset = Asset() 
    weekly_path = my_mc_asset.get_weekly_path()
    print("weekly_path = {}, workflow_id = {}".format(weekly_path, my_mc_asset.workflow_id))
    save_path = os.path.join(weekly_path, my_mc_asset.workflow_id)
    print("save_path = {}".format(save_path))

    HOME = os.environ["HOME"]
    print("\nHOME = {}".format(HOME))
    #running_path = os.getcwd()
    running_path = os.path.join(HOME, 'scripts')
    print("\nrunning_path = {}".format(running_path))
    neo4j_path = "neo4j-community-4.4.5"
    neo4j_import_path = os.path.join(neo4j_path, 'import')
    neo4j_conf_path = os.path.join(neo4j_path, 'conf')
    neo4j_bin_path = os.path.join(neo4j_path, 'bin')
    neo4j_running_import_path = os.path.join(running_path, neo4j_import_path)
    neo4j_running_conf_path = os.path.join(running_path, neo4j_conf_path)
    neo4j_running_bin_path = os.path.join(running_path, neo4j_bin_path)
    DATE = datetime.today().strftime("%Y_%m_%d")
    print("\nDATE = {}".format(DATE))
else:
    HOME = os.environ["HOME"]
    print("\nHOME = {}".format(HOME))
    #running_path = os.getcwd()
    running_path = os.path.join(HOME, 'ssd/projects/GraphDB/customer_index/complaint_index')
    print("\nrunning_path = {}".format(running_path))
    save_path = os.path.join(running_path,"data")
    print("\nsave_path = {}".format(save_path))
    neo4j_path = "ssd/projects/GraphDB/neo4j/neo4j-community-4.4.4"
    neo4j_import_path = os.path.join(neo4j_path, 'import')
    neo4j_conf_path = os.path.join(neo4j_path, 'conf')
    neo4j_bin_path = os.path.join(neo4j_path, 'bin')
    neo4j_running_import_path = os.path.join(HOME, neo4j_import_path)
    neo4j_running_conf_path = os.path.join(HOME, neo4j_conf_path)
    neo4j_running_bin_path = os.path.join(HOME, neo4j_bin_path)
    DATE = datetime.today().strftime("%Y_%m_%d")
    print("\nDATE = {}".format(DATE))


HOME = /home/sookyoung.kim

running_path = /home/sookyoung.kim/ssd/projects/GraphDB/customer_index/complaint_index

save_path = /home/sookyoung.kim/ssd/projects/GraphDB/customer_index/complaint_index/data

DATE = 2023_11_14


In [33]:
# Port, ID, Passwd 설정
if this_env == "MC":
    n4j_bolt_port = 7687
else:
    # You can use 7687 also for local. But I use 7689 just to avoid the collision to other version of neo4j on the same machine.
    n4j_bolt_port = 7689  
n4j_id = "neo4j"
n4j_pwd = "neo4j"  # Initial default password

만일 아래 코드 수행 시 error 발생하면 몇 번 다시 시도해 볼 것

In [34]:
if this_env == "MC":
    # neo4j.conf 내용 확인
    print("\nCheck neo4j.conf-----------------------------------------------------")
    print(os.system("cat /home/splunk/scripts/neo4j-community-4.4.5/conf/neo4j.conf"))
    print("-----------------------------------------------------------------------")

    # inference.py 내용 확인
    print("\nCheck inference.py---------------------------------------")
    print(os.system("cat /home/splunk/scripts/inference.py"))
    print("-----------------------------------------------------------------------")

    # neo4j stop - 이전 수행 결과로 7687 port가 occupied되어 있을 가능성 제거 위해
    print("\nStop neo4j-----------------------------------------------------------")
    os.system("/home/splunk/scripts/neo4j-community-4.4.5/bin/neo4j stop")
    print("-----------------------------------------------------------------------")

    # neo4j start
    print("\nStart neo4j----------------------------------------------------------")
    os.system("/home/splunk/scripts/neo4j-community-4.4.5/bin/neo4j start --verbose")
    os.system("sleep 30")
    print("-----------------------------------------------------------------------")

    # Password change
    #os.system('/home/splunk/scripts/neo4j-community-4.4.5/bin/neo4j-admin set-initial-password neo4jmc')
    if(n4j_pwd=="neo4j"):
        print("\nPassword change to neo4jmc-------------------------------------------")
        os.system('/home/splunk/scripts/neo4j-community-4.4.5/bin/neo4j-admin set-initial-password neo4jmc')
        n4j_pwd = "neo4jmc"
        print("-----------------------------------------------------------------------")

    # save_path의 cmdt_table_*.csv, vendor_table_*.csv, cntr_table_*.csv, cypher_query_list.pkl를 neo4j_running_import_path로 복사
    print("\nCopy cmdt_table_*.csv, vendor_table_*.csv, cntr_table_*.csv, cypher_query_list.pkl from {} to {}".format(save_path, neo4j_running_import_path))
    os.system(f"cp {save_path}/cmdt_table_*.csv {neo4j_running_import_path}/")
    os.system(f"cp {save_path}/vendor_table_*.csv {neo4j_running_import_path}/")
    os.system(f"cp {save_path}/cntr_table_*.csv {neo4j_running_import_path}/")
    os.system(f"cp {save_path}/cypher_query_list.pkl {neo4j_running_import_path}/")
    print("-----------------------------------------------------------------------")

    # Neo4j file들의 owner 확인
    print("\nCheck file owners----------------------------------------------------")
    print(os.system("ls -al /home/splunk/scripts/neo4j-community-4.4.5/*/*"))
    print("-----------------------------------------------------------------------")

    # Container의 network 상태 확인
    print("\nChecking container network status -----------------------------------")
    # netstat -lnp
    print("netstat -lnp")
    print(os.system("netstat -lnp"))
    # ping
    print("ping -c 3 ...")
    print(os.system("ping -c 3 localhost"))
    print(os.system("ping -c 3 127.0.0.1"))
    # ifconfig
    print("ifconfig")
    print(os.system("ifconfig"))
    # curl
    print("curl")
    print(os.system("curl http://localhost:7474"))
    print(os.system("curl bolt://localhost:7687"))
    print(os.system("curl http://127.0.0.1:7474"))
    print(os.system("curl bolt://127.0.0.1:7687"))
    # lsof
    print("lsof")
    print(os.system("lsof -i -P -n"))
    print("-----------------------------------------------------------------------")
else:
    import subprocess
    
    if 0:
        # neo4j.conf 내용 확인
        print("\nCheck neo4j.conf-----------------------------------------------------")
        cmd = "cat"
        param = os.path.join(str(neo4j_running_conf_path),"neo4j.conf")
        result = subprocess.run([cmd,param], capture_output=True, universal_newlines=True)
        print(result.stdout)
        print("-----------------------------------------------------------------------")
    
    if 0:
        # inference.py 내용 확인
        print("\nCheck inference.py---------------------------------------")
        cmd = "cat"
        param = os.path.join(str(running_path),"inference.py")
        result = subprocess.run([cmd,param], capture_output=True, universal_newlines=True)
        print(result.stdout)
        print("-----------------------------------------------------------------------")

    # neo4j stop - 이전 수행 결과로 7687 port가 occupied되어 있을 가능성 제거 위해
    print("\nStop neo4j-----------------------------------------------------------")
    cmd = os.path.join(str(neo4j_running_bin_path),"neo4j")
    param = "stop"
    result = subprocess.run([cmd,param], capture_output=True, universal_newlines=True)
    print(result.stdout)
    print("-----------------------------------------------------------------------")

    # neo4j start
    print("\nStart neo4j----------------------------------------------------------")
    cmd = os.path.join(str(neo4j_running_bin_path),"neo4j")
    param1 = "start"
    param2 = "--verbose"
    result = subprocess.run([cmd,param1,param2], capture_output=True, universal_newlines=True)
    print(result.stdout)
    cmd = "sleep"
    param = "3"
    result = subprocess.run([cmd,param], capture_output=True, universal_newlines=True)
    print(result.stdout)
    print("-----------------------------------------------------------------------")

    # Password change
    #os.system('/home/splunk/scripts/neo4j-community-4.4.5/bin/neo4j-admin set-initial-password neo4jmc')
    if(n4j_pwd=="neo4j"):
        print("\nPassword change to neo4jmc-------------------------------------------")
        cmd = os.path.join(str(neo4j_running_bin_path),"neo4j-admin")
        param1 = "set-initial-password"
        param2 = "neo4jmc"
        result = subprocess.run([cmd,param1,param2], capture_output=True, universal_newlines=True)
        print(result.stdout)
        n4j_pwd = "neo4jmc"
        print("-----------------------------------------------------------------------")

    # save_path의 node_*.csv를 neo4j_running_import_path로 복사
    print("\nCopy node_*.csv from {} to {}".format(save_path, neo4j_running_import_path))
    os.system(f"cp {save_path}/node_*.csv {neo4j_running_import_path}/")
    #os.system(f"cp {save_path}/cypher_query_list.pkl {neo4j_running_import_path}/")
    print("-----------------------------------------------------------------------")

    if 0:
        # Neo4j file들의 owner 확인
        print("\nCheck file owners----------------------------------------------------")
        print(os.system("ls -al /home/splunk/scripts/neo4j-community-4.4.5/*/*"))
        cmd = "ls"
        param1 = "-al"
        param2 = os.path.join(str(neo4j_running_bin_path), "..")
        result = subprocess.run([cmd,param1,param2], capture_output=True, universal_newlines=True)
        print(result.stdout)
        print("-----------------------------------------------------------------------")

    # Container의 network 상태 확인
    print("\nChecking container network status -----------------------------------")
    if 1:
        # netstat -lnp
        print("netstat -lnp")
        cmd = "netstat"
        param = "-lnp"
        result = subprocess.run([cmd,param], capture_output=True, universal_newlines=True)
        print(result.stdout)
    if 0:
        # ping
        print("ping -c 3 ...")
        cmd = "ping"
        param1 = "-c"
        param2 = "3"
        param3 = "localhost"
        result = subprocess.run([cmd,param1,param2,param3], capture_output=True, universal_newlines=True)
        print(result.stdout)
        cmd = "ping"
        param1 = "-c"
        param2 = "3"
        param3 = "127.0.0.1"
        result = subprocess.run([cmd,param1,param2,param3], capture_output=True, universal_newlines=True)
        print(result.stdout)
    if 0:
        # ifconfig
        print("ifconfig")
        cmd = "ifconfig"
        result = subprocess.run([cmd,], capture_output=True, universal_newlines=True)
        print(result.stdout)
    if 0:
        # curl
        print("curl")
        print(os.system("curl http://localhost:7474"))
        print(os.system("curl bolt://localhost:7689"))
        print(os.system("curl http://127.0.0.1:7474"))
        print(os.system("curl bolt://127.0.0.1:7689"))
    if 0:
        # lsof
        print("lsof")
        cmd = "lsof"
        param1 = "-i"
        param2 = "-P"
        param3 = "-n"
        result = subprocess.run([cmd,param1,param2,param3], capture_output=True, universal_newlines=True)
        print(result.stdout)
    print("-----------------------------------------------------------------------")


Stop neo4j-----------------------------------------------------------
Neo4j is not running.

-----------------------------------------------------------------------

Start neo4j----------------------------------------------------------
Directories in use:
home:         /home/sookyoung.kim/ssd/projects/GraphDB/neo4j/neo4j-community-4.4.4
config:       /home/sookyoung.kim/ssd/projects/GraphDB/neo4j/neo4j-community-4.4.4/conf
logs:         /home/sookyoung.kim/ssd/projects/GraphDB/neo4j/neo4j-community-4.4.4/logs
plugins:      /home/sookyoung.kim/ssd/projects/GraphDB/neo4j/neo4j-community-4.4.4/plugins
import:       /home/sookyoung.kim/ssd/projects/GraphDB/neo4j/neo4j-community-4.4.4/import
data:         /home/sookyoung.kim/ssd/projects/GraphDB/neo4j/neo4j-community-4.4.4/data
certificates: /home/sookyoung.kim/ssd/projects/GraphDB/neo4j/neo4j-community-4.4.4/certificates
licenses:     /home/sookyoung.kim/ssd/projects/GraphDB/neo4j/neo4j-community-4.4.4/licenses
run:          /home/sookyou

# 3. Neo4j 원격 접속 class 선언 및 test

In [35]:
class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            #self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd), max_transaction_retry_time=300)
            #self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd), max_transaction_retry_time=300, encrypted=False)
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def verify_connectivity(self):
        if self.__driver is not None:
            print(self.__driver.verify_connectivity())

    def supports_multi_db(self):
        if self.__driver is not None:
            print(self.__driver.supports_multi_db())
    def default_host(self):
        if self.__driver is not None:
            return self.__driver.default_host
    def default_port(self):
        if self.__driver is not None:
            return self.__driver.default_port
    def encrypted(self):
        if self.__driver is not None:
            return self.__driver.encrypted
            
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized"
        session = None
        response = None
        try:
            #session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            session = self.__driver.session(database=DEFAULT_DATABASE, default_access_mode=WRITE_ACCESS)
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

## Neo4j connection test  

- 주어진 ADDRESS, PORT, id, pwd로 연결되는지, 연결 verification 정상 동작하는지 확인 후 close  
- 이를 통해 오류 발생 시 connection 문제인지 확인 가능

In [26]:
if 0:
    ADDRESS = "localhost"
    PORT = n4j_bolt_port # 정적 PORT
    id = n4j_id
    pwd = n4j_pwd
    print(ADDRESS, PORT, id, pwd)

    print("Test begins")
    conn = Neo4jConnection(uri=f"bolt://{ADDRESS}:{PORT}", user= id , pwd= pwd)
    #conn = Neo4jConnection(uri="bolt://localhost:7689", user="neo4j", pwd="neo4jmc")
    print("Connected with ", conn)

    print("Connection can be verified if Neo4j/4.4.4 is returned")
    conn.verify_connectivity()

    print("Connection for multiple DBs?")
    conn.supports_multi_db()

    conn.close()
    print("Test completed")

# 4. Neo4j connect

만일 아래 코드 수행 시 error 발생하면 몇 번 다시 시도해 볼 것

In [36]:
ADDRESS = "localhost"
PORT = n4j_bolt_port # 정적 PORT
id = n4j_id
pwd = n4j_pwd
print("ADDRESS = {}, PORT = {}, id = {}, pwd = {}".format(ADDRESS, PORT, id, pwd))

print("\nConnect to neo4j-----------------------------------------------------")
conn = Neo4jConnection(uri=f"bolt://{ADDRESS}:{PORT}", user= id , pwd= pwd)
print("Connected with ", conn)

print("\nCheck neo4j connection-----------------------------------------------")
conn.verify_connectivity()
conn.supports_multi_db()
print("\conn.default_host = {}".format(conn.default_host()))
print("\conn.default_port = {}".format(conn.default_port()))
print("\conn.encrypted = {}".format(conn.encrypted()))
print("-----------------------------------------------------------------------")

ADDRESS = localhost, PORT = 7689, id = neo4j, pwd = neo4jmc

Connect to neo4j-----------------------------------------------------
Connected with  <__main__.Neo4jConnection object at 0x7fcb6ebbf690>

Check neo4j connection-----------------------------------------------
None
True
\conn.default_host = localhost
\conn.default_port = 7687
\conn.encrypted = False
-----------------------------------------------------------------------


## 원격 서버에 대한 query test

위 neo4j connection을 한 상태에서 query 처리가 정상인지 test

In [17]:
def create_node(conn, name_to_create):
    conn.query("merge (:test_node {name: " f"{name_to_create}" "})", db = DEFAULT_DATABASE)

def find_all_nodes(conn):
    conn.query("match (n) return *", db = DEFAULT_DATABASE)

def find_nodes(conn, node_type):
    cypher_query = \
    f'''
    MATCH (n:{node_type})
    RETURN *
    '''
    result = conn.query(cypher_query, db = DEFAULT_DATABASE)
    print(result)

def update_property(conn, node_type, node_name, property_name, property_value):
    cypher_query = \
        "MERGE (n:" f"{node_type}" "{" f"name: {node_name}" "}) " \
        f"ON CREATE SET n.{property_name} = {property_value} " \
        f"ON MATCH SET n.{property_name} = {property_value}"
    
    conn.query(cypher_query, db = DEFAULT_DATABASE)

def connect_edge(conn, node_type1, node_type2, key1, key2, relation_type):
    cypher_query = \
        "MATCH (n : " f"{node_type1}" "), (m: " f"{node_type2}" ") " \
        "WHERE n." f"{key1}" " = m." f"{key2} " \
        "CREATE (n)-[:" f"{relation_type}" "]->(m)"
    
    conn.query(cypher_query, db = DEFAULT_DATABASE)    

def delete_nodes(conn, node_type):
    cypher_query = \
        f'''
        MATCH (n: {node_type})
        DETACH
        DELETE n
        '''
    conn.query(cypher_query, db = DEFAULT_DATABASE)
    
def delete_all_nodes(conn):
    conn.query("match (n) detach delete n", db = DEFAULT_DATABASE)

In [11]:
create_node(conn, '"test_node1"')
create_node(conn, '"test_node2"')
create_node(conn, '"test_node3"')

In [53]:
find_nodes(conn, "test_node")

[<Record n=<Node id=4975482 labels=frozenset({'test_node'}) properties={'prev': 3.0, 'name': 'test_node1', 'curr': 1.0}>>, <Record n=<Node id=4975483 labels=frozenset({'test_node'}) properties={'prev': 1.0, 'name': 'test_node2', 'curr': 2.0}>>, <Record n=<Node id=4975484 labels=frozenset({'test_node'}) properties={'prev': 2.0, 'name': 'test_node3', 'curr': 3.0}>>]


In [47]:
update_property(conn, 'test_node', '"test_node1"', 'curr', '1.0')
update_property(conn, 'test_node', '"test_node2"', 'curr', '2.0')
update_property(conn, 'test_node', '"test_node3"', 'curr', '3.0')

In [50]:
update_property(conn, 'test_node', '"test_node1"', 'prev', '3.0')
update_property(conn, 'test_node', '"test_node2"', 'prev', '1.0')
update_property(conn, 'test_node', '"test_node3"', 'prev', '2.0')

In [58]:
connect_edge(conn, 'test_node', 'test_node', 'prev', 'curr', 'rel_prev')

In [ ]:
find_nodes(conn, 'test_node')

In [61]:
delete_nodes(conn, 'test_node')

아래는 위의 함수화 된 query들과 같은 기능들

In [96]:
test_run = 0

In [97]:
if test_run:
    print(DEFAULT_DATABASE)

In [98]:
if test_run:
    cypher_query = \
    "merge (:test_node {name: 'test_node1'})"

    conn.query(cypher_query, db = DEFAULT_DATABASE)

In [146]:
if test_run:
    cypher_query = \
    "match (n) return *"

    conn.query(cypher_query, db = DEFAULT_DATABASE)

In [147]:
if test_run:
    cypher_query = \
    "match (n) detach delete n"

    conn.query(cypher_query, db = DEFAULT_DATABASE)

원격 접속을 통한 query 수행이 끝나면 connection을 닫는다

In [57]:
conn.close()

# 5. Graph 구성 (node 생성, edge 생성)

## 5.1. Nodes 생성

### 5.1.1. CX node 생성

CX node는 다른 node들과 연결하기 위한 key 역할의 properties를 포함, 많은 properties를 가지고 있으므로 manually 생성해준다.

In [37]:
# Create CX nodes
# CX nodes have a set of custom properites. So, we create it manually.

cypher_query = \
    '''
    LOAD CSV WITH HEADERS FROM "file:///node_CX.csv" AS row
    CREATE (label_CX:CX)
    SET label_CX = row
    SET label_CX.CX_ID = row.CX_ID
    SET label_CX.CX_TYPE = toInteger(row.CX_TYPE)
    SET label_CX.ETL_LOAD_DATE = toInteger(row.ETL_LOAD_DATE)
    SET label_CX.RCPT_DTHM = toInteger(row.RCPT_DTHM)
    SET label_CX.CX_ID_PREV = row.CX_ID_PREV
    SET label_CX.LABEL_CLAIM_NEXT = toInteger(row.LABEL_CLAIM_NEXT)
    SET label_CX.LABEL_CLAIM_FUTURE = toInteger(row.LABEL_CLAIM_FUTURE)
    SET label_CX.LABEL_CLAIM = toInteger(row.LABEL_CLAIM)
    SET label_CX.LABEL_CLAIM_INF = toInteger(row.LABEL_CLAIM_INF)
    SET label_CX.FLAG_TRAIN_INFER = toInteger(row.FLAG_TRAIN_INFER)
    SET label_CX.CUST_NO = row.CUST_NO
    SET label_CX.CUST_TYPE = row.CUST_TYPE
    SET label_CX.VIP_CUST_TYPE = row.VIP_CUST_TYPE
    SET label_CX.VIP_GRADE = row.VIP_GRADE
    SET label_CX.SERIAL_NO = row.SERIAL_NO
    SET label_CX.MODEL_CODE = row.MODEL_CODE
    SET label_CX.PRODUCT_CODE = row.PRODUCT_CODE
    SET label_CX.CO_RCPT_CHANNEL = row.CO_RCPT_CHANNEL
    SET label_CX.CO_RCPT_USER_ID = row.CO_RCPT_USER_ID
    SET label_CX.CO_RCPT_DEPT_CODE = row.CO_RCPT_DEPT_CODE
    SET label_CX.CO_RCPT_WTY_TYPE = row.CO_RCPT_WTY_TYPE
    SET label_CX.CO_COUNS_LARGE_CLASS = row.CO_COUNS_LARGE_CLASS
    SET label_CX.CO_COUNS_MID_CLASS = row.CO_COUNS_MID_CLASS
    SET label_CX.CO_COUNS_SMALL_CLASS = row.CO_COUNS_SMALL_CLASS
    SET label_CX.CO_COUNS_STAT = row.CO_COUNS_STAT
    SET label_CX.RP_RCPT_DEPT_TYPE = row.RP_RCPT_DEPT_TYPE
    SET label_CX.RP_RCPT_USER_ID = row.RP_RCPT_USER_ID
    SET label_CX.RP_REPAIR_SE_ID = row.RP_REPAIR_SE_ID
    SET label_CX.RP_RCPT_TYPE = row.RP_RCPT_TYPE
    SET label_CX.RP_RCPT_WTY_TYPE = row.RP_RCPT_WTY_TYPE
    SET label_CX.RP_RCPT_SYMP_CODE = row.RP_RCPT_SYMP_CODE
    SET label_CX.RP_RCPT_SUB_SYMP_CODE = row.RP_RCPT_SUB_SYMP_CODE
    SET label_CX.RP_FAIL_CODE_GROUP = row.RP_FAIL_CODE_GROUP
    SET label_CX.RP_FAIL_LARGE_CLASS = row.RP_FAIL_LARGE_CLASS
    SET label_CX.RP_FAIL_MID_CLASS = row.RP_FAIL_MID_CLASS
    SET label_CX.RP_FAIL_REASON_CODE = row.RP_FAIL_REASON_CODE
    SET label_CX.RP_FAIL_TYPE = row.RP_FAIL_TYPE
    SET label_CX.RP_REPAIR_WTY_TYPE = row.RP_REPAIR_WTY_TYPE
    SET label_CX.RP_TOTAL_FEE_AMT = row.RP_TOTAL_FEE_AMT
    SET label_CX.RP_TOTAL_AMT = row.RP_TOTAL_AMT
    SET label_CX.RP_STATUS = row.RP_STATUS
    SET label_CX.RP_UNSATIS_ETD_TYPE1 = row.RP_UNSATIS_ETD_TYPE1
    SET label_CX.RP_UNSATIS_ETD_TYPE2 = row.RP_UNSATIS_ETD_TYPE2
    SET label_CX.RP_RE_REPAIR_TYPE = row.RP_RE_REPAIR_TYPE
    SET label_CX.EX_AGENT_CODE = row.EX_AGENT_CODE
    SET label_CX.EX_CAUSE_CODE = row.EX_CAUSE_CODE
    SET label_CX.EX_BUY_FACTOR_CODE = row.EX_BUY_FACTOR_CODE
    SET label_CX.EX_EXCHANGE_ORDER_TYPE = row.EX_EXCHANGE_ORDER_TYPE
    SET label_CX.EX_COMPLETE_YN = row.EX_COMPLETE_YN
    SET label_CX.CL_CLAIM_RCPT_CHANNEL = row.CL_CLAIM_RCPT_CHANNEL
    SET label_CX.CL_CLAIM_RCPT_TYPE = row.CL_CLAIM_RCPT_TYPE
    SET label_CX.CL_RCPT_DEPT_CODE = row.CL_RCPT_DEPT_CODE
    SET label_CX.CL_RCPT_USER_ID = row.CL_RCPT_USER_ID
    SET label_CX.CL_CLAIM_TYPE = row.CL_CLAIM_TYPE
    SET label_CX.CL_CLAIM_LARGE_CLASS = row.CL_CLAIM_LARGE_CLASS
    SET label_CX.CL_CLAIM_MID_CLASS = row.CL_CLAIM_MID_CLASS
    SET label_CX.CL_CLAIM_SMALL_CLASS = row.CL_CLAIM_SMALL_CLASS
    SET label_CX.CL_TGET_SECTION = row.CL_TGET_SECTION
    SET label_CX.CL_TGET_DEPT_CODE = row.CL_TGET_DEPT_CODE
    SET label_CX.CL_CUST_REQ_TYPE = row.CL_CUST_REQ_TYPE
    SET label_CX.CL_PRCS_TYPE = row.CL_PRCS_TYPE
    SET label_CX.CL_CLAIM_WTY_TYPE = row.CL_CLAIM_WTY_TYPE
    SET label_CX.CL_CLAIM_STATUS = row.CL_CLAIM_STATUS
    SET label_CX.CX_CNT = toInteger(row.CX_CNT)
    RETURN label_CX
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[<Record label_CX=<Node element_id='0' labels=frozenset({'CX'}) properties={'CO_RCPT_CHANNEL': 'A', 'RCPT_DTHM': 20230127110400, 'CO_RCPT_WTY_TYPE': 'Y', 'CO_RCPT_USER_ID': 'UB04510', 'LABEL_CLAIM_INF': 0, 'CX_CNT': 3, 'CO_COUNS_STAT': 'Y', 'FLAG_TRAIN_INFER': 2, 'CX_TYPE': 0, 'CUST_NO': 'C000002735', 'ETL_LOAD_DATE': 20230128035021, 'CUST_TYPE': 'E', 'PRODUCT_CODE': 'LCL', 'CX_ID_PREV': 'MC061152CK', 'LABEL_CLAIM_FUTURE': 0, 'CO_COUNS_LARGE_CLASS': 'CL01', 'CO_COUNS_MID_CLASS': 'CM06', 'LABEL_CLAIM': 0, 'CX_ID': 'J052975145', 'CO_COUNS_SMALL_CLASS': 'CS01', 'CO_RCPT_DEPT_CODE': '12456', 'MODEL_CODE': '65UM7900BNA.AKRYLH', 'LABEL_CLAIM_NEXT': 0}>>,
 <Record label_CX=<Node element_id='1' labels=frozenset({'CX'}) properties={'CO_RCPT_CHANNEL': 'A', 'RCPT_DTHM': 20230127113400, 'CO_RCPT_WTY_TYPE': 'Y', 'CO_RCPT_USER_ID': 'UB04733', 'LABEL_CLAIM_INF': 0, 'CX_CNT': 3, 'CO_COUNS_STAT': 'Z', 'FLAG_TRAIN_INFER': 2, 'CX_TYPE': 0, 'CUST_NO': 'C000002735', 'ETL_LOAD_DATE': 20230128035035, 'CUST_T

### 5.1.2. CX_TYPE node 생성

다음과 같은 이유로 CX_TYPE node도 manually 생성한다. 

생성된 original graph에 대해 특정 node를 제외하고 embedding하고자 할 때 sub-graph를 projection해서 embdding하게 되는데,  
sub-graph를 만들고자 특정 node를 filtering하는 방법은 보통 node의 property 값을 조건으로 사용하게 된다. 

그런데, neo4j community edition은 monopartite graph만 embedding 가능하므로  
(더 정확하게는 embedding 전 단계인 projection 시 monopartite graph만 가능하다.  
Neo4j의 embedding은 projection되어 memory에 올라간 graph를 입력으로 쓴다),  

생성된 graph의 각 type의 nodes는 서로 다른 property set을 가지더라도  
projection 단계에서는 모든 nodes가 공통적으로 가지는 properties만을 포함시킬 수 있다.  

불만지수의 경우 sub-graph 생성 시 CX_TYPE을 이용하는 경우가 많으므로,  
CX_TYPE은 모든 type의 nodes가 property로 가지도록 해야 하는데,   
CX_TYPE node가 아닌 경우는 CX_TYPE property를 default값으로 주도록 작성되어 있다.

따라서 만일 CX_TYPE node도 iterative한 node 생성 코드로 돌리게 되면 CX_TYPE node의 CX_TYPE property도 무조건 default값이 되는 오류가 발생.

결론적으로, 나중에 node filtering에 사용할 property와 같은 이름을 갖는 node type은 별도로 생성해야 한다.

In [38]:
# Create CX_TYPE nodes
# CX nodes have a set of custom properites. So, we create it manually.

cypher_query = \
    '''
    LOAD CSV WITH HEADERS FROM "file:///node_CX_TYPE.csv" AS row
    CREATE (label_CX_TYPE:CX_TYPE)
    SET label_CX_TYPE = row
    SET label_CX_TYPE.CX_TYPE = toInteger(row.CX_TYPE)
    SET label_CX_TYPE.RCPT_DTHM = toInteger("10000101000000")
    SET label_CX_TYPE.LABEL_CLAIM_NEXT = toInteger(row.LABEL_CLAIM_NEXT)
    SET label_CX_TYPE.LABEL_CLAIM_FUTURE = toInteger(row.LABEL_CLAIM_FUTURE)
    SET label_CX_TYPE.LABEL_CLAIM = toInteger(row.LABEL_CLAIM)
    SET label_CX_TYPE.LABEL_CLAIM_INF = toInteger(row.LABEL_CLAIM_INF)
    SET label_CX_TYPE.FLAG_TRAIN_INFER = toInteger(row.FLAG_TRAIN_INFER)
    RETURN label_CX_TYPE
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[<Record label_CX_TYPE=<Node element_id='1224979' labels=frozenset({'CX_TYPE'}) properties={'RCPT_DTHM': 10000101000000, 'LABEL_CLAIM_INF': 0, 'LABEL_CLAIM_FUTURE': 0, 'FLAG_TRAIN_INFER': 2, 'CX_TYPE': 0, 'LABEL_CLAIM': 0, 'LABEL_CLAIM_NEXT': 0}>>,
 <Record label_CX_TYPE=<Node element_id='1224980' labels=frozenset({'CX_TYPE'}) properties={'RCPT_DTHM': 10000101000000, 'LABEL_CLAIM_INF': 0, 'LABEL_CLAIM_FUTURE': 0, 'FLAG_TRAIN_INFER': 2, 'CX_TYPE': 1, 'LABEL_CLAIM': 0, 'LABEL_CLAIM_NEXT': 0}>>,
 <Record label_CX_TYPE=<Node element_id='1224981' labels=frozenset({'CX_TYPE'}) properties={'RCPT_DTHM': 10000101000000, 'LABEL_CLAIM_INF': 0, 'LABEL_CLAIM_FUTURE': 0, 'FLAG_TRAIN_INFER': 1, 'CX_TYPE': 2, 'LABEL_CLAIM': 0, 'LABEL_CLAIM_NEXT': 0}>>,
 <Record label_CX_TYPE=<Node element_id='1224982' labels=frozenset({'CX_TYPE'}) properties={'RCPT_DTHM': 10000101000000, 'LABEL_CLAIM_INF': 0, 'LABEL_CLAIM_FUTURE': 0, 'FLAG_TRAIN_INFER': 2, 'CX_TYPE': 3, 'LABEL_CLAIM': 0, 'LABEL_CLAIM_NEXT': 0}>>]

In [39]:
# Create CUST_NO nodes
# CUST_NO nodes have a set of custom properites. So, we create it manually.

cypher_query = \
    '''
    LOAD CSV WITH HEADERS FROM "file:///node_CUST_NO.csv" AS row
    CREATE (label_CUST_NO:CUST_NO)
    SET label_CUST_NO = row
    SET label_CUST_NO.CUST_NO = row.CUST_NO
    SET label_CUST_NO.CX_TYPE = toInteger('9')
    SET label_CUST_NO.RCPT_DTHM = toInteger("10000101000000")
    SET label_CUST_NO.LABEL_CLAIM_NEXT = toInteger(row.LABEL_CLAIM_NEXT)
    SET label_CUST_NO.LABEL_CLAIM_FUTURE = toInteger(row.LABEL_CLAIM_FUTURE)
    SET label_CUST_NO.LABEL_CLAIM = toInteger(row.LABEL_CLAIM)
    SET label_CUST_NO.LABEL_CLAIM_INF = toInteger(row.LABEL_CLAIM_INF)
    SET label_CUST_NO.FLAG_TRAIN_INFER = toInteger(row.FLAG_TRAIN_INFER)
    RETURN label_CUST_NO
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[<Record label_CUST_NO=<Node element_id='1224983' labels=frozenset({'CUST_NO'}) properties={'RCPT_DTHM': 10000101000000, 'LABEL_CLAIM_INF': 0, 'LABEL_CLAIM_FUTURE': 0, 'FLAG_TRAIN_INFER': 2, 'CX_TYPE': 9, 'CUST_NO': 'C000002735', 'LABEL_CLAIM': 0, 'LABEL_CLAIM_NEXT': 0}>>,
 <Record label_CUST_NO=<Node element_id='1224984' labels=frozenset({'CUST_NO'}) properties={'RCPT_DTHM': 10000101000000, 'LABEL_CLAIM_INF': 0, 'LABEL_CLAIM_FUTURE': 0, 'FLAG_TRAIN_INFER': 2, 'CX_TYPE': 9, 'CUST_NO': 'C000002881', 'LABEL_CLAIM': 0, 'LABEL_CLAIM_NEXT': 0}>>,
 <Record label_CUST_NO=<Node element_id='1224985' labels=frozenset({'CUST_NO'}) properties={'RCPT_DTHM': 10000101000000, 'LABEL_CLAIM_INF': 0, 'LABEL_CLAIM_FUTURE': 0, 'FLAG_TRAIN_INFER': 2, 'CX_TYPE': 9, 'CUST_NO': 'C000003470', 'LABEL_CLAIM': 0, 'LABEL_CLAIM_NEXT': 0}>>,
 <Record label_CUST_NO=<Node element_id='1224986' labels=frozenset({'CUST_NO'}) properties={'RCPT_DTHM': 10000101000000, 'LABEL_CLAIM_INF': 0, 'LABEL_CLAIM_FUTURE': 0, 'FLAG_TRAI

### 5.1.2. CX와 CX_TYPE, CUST_NO 외 나머지 node들 생성

CX와 CX_TYPE 외의 node들은 CX node와의 연결을 위한 key 역할을 하는 property 1개와 graph projection을 위해 CX와 맞춰주기 위한 형식적 properties를 공통적으로 가지므로 iterative하게 생성함.

In [40]:
## 고객 정보 nodes, 제품 정보 nodes, 고객 경험 정보 nodes
list_node_name = [
    #'CX_TYPE',  # 별도 생성
    'CUST_TYPE',
    'VIP_CUST_TYPE',
    'VIP_GRADE',
    'MODEL_CODE',
    'PRODUCT_CODE',
    'CO_RCPT_CHANNEL',
    'CO_RCPT_USER_ID',
    'CO_RCPT_DEPT_CODE',
    'CO_RCPT_WTY_TYPE',
    'CO_COUNS_LARGE_CLASS',
    'CO_COUNS_MID_CLASS',
    'CO_COUNS_SMALL_CLASS',
    'CO_COUNS_STAT',
    'RP_RCPT_DEPT_TYPE',
    'RP_RCPT_USER_ID',
    'RP_REPAIR_SE_ID',
    'RP_RCPT_TYPE',
    'RP_RCPT_WTY_TYPE',
    'RP_RCPT_SYMP_CODE',
    'RP_RCPT_SUB_SYMP_CODE',
    'RP_FAIL_CODE_GROUP',
    'RP_FAIL_LARGE_CLASS',
    'RP_FAIL_MID_CLASS',
    'RP_FAIL_REASON_CODE',
    'RP_FAIL_TYPE',
    'RP_REPAIR_WTY_TYPE',
    'RP_TOTAL_FEE_AMT',
    'RP_TOTAL_AMT',
    'RP_STATUS',
    'RP_UNSATIS_ETD_TYPE1',
    'RP_UNSATIS_ETD_TYPE2',
    'RP_RE_REPAIR_TYPE',
    'EX_AGENT_CODE',
    'EX_CAUSE_CODE',
    'EX_BUY_FACTOR_CODE',
    'EX_EXCHANGE_ORDER_TYPE',
    'EX_COMPLETE_YN',
    'CL_CLAIM_RCPT_CHANNEL',
    'CL_CLAIM_RCPT_TYPE',
    'CL_RCPT_DEPT_CODE',
    'CL_RCPT_USER_ID',
    'CL_CLAIM_TYPE',
    'CL_CLAIM_LARGE_CLASS',
    'CL_CLAIM_MID_CLASS',
    'CL_CLAIM_SMALL_CLASS',
    #'CL_SYMP_CODE',  # 전부 Null
    #'CL_DETAIL_SYMP_CODE',  # 전부 Null
    'CL_TGET_SECTION',
    'CL_TGET_DEPT_CODE',
    'CL_CUST_REQ_TYPE',
    'CL_PRCS_TYPE',
    'CL_CLAIM_WTY_TYPE',
    'CL_CLAIM_STATUS',
    #'CUST_NO',
    'SERIAL_NO',
    #'CL_SATIS_TYPE',  # 전부 Null
]

In [41]:
def create_label_nodes_from_csv(conn, node_name):
    
    node_label_name = "label_"+node_name
    property_name = node_name

    cypher_query = \
        f'''
        LOAD CSV WITH HEADERS FROM "file:///node_{node_name}.csv" AS row
        CREATE ({node_label_name}:{node_name})
        SET {node_label_name} = row
        SET {node_label_name}.{property_name} = row.{property_name}
        // 이하는 graph projection 시 CX node의 해당 properties를 projection하기 위한 dummy
        SET {node_label_name}.CX_TYPE = toInteger('9')
        SET {node_label_name}.RCPT_DTHM = toInteger("10000101000000")
        SET {node_label_name}.LABEL_CLAIM_NEXT = toInteger(row.LABEL_CLAIM_NEXT)
        SET {node_label_name}.LABEL_CLAIM_FUTURE = toInteger(row.LABEL_CLAIM_FUTURE)
        SET {node_label_name}.LABEL_CLAIM = toInteger(row.LABEL_CLAIM)
        SET {node_label_name}.LABEL_CLAIM_INF = toInteger(row.LABEL_CLAIM_INF)
        SET {node_label_name}.FLAG_TRAIN_INFER = toInteger(row.FLAG_TRAIN_INFER)
        RETURN {node_label_name}
        '''

    conn.query(cypher_query, db = DEFAULT_DATABASE)

In [42]:
for nm in list_node_name:
    create_label_nodes_from_csv(conn, nm)
    print("Nodes {} has been added".format(nm))

Nodes CUST_TYPE has been added
Nodes VIP_CUST_TYPE has been added
Nodes VIP_GRADE has been added
Nodes MODEL_CODE has been added
Nodes PRODUCT_CODE has been added
Nodes CO_RCPT_CHANNEL has been added
Nodes CO_RCPT_USER_ID has been added
Nodes CO_RCPT_DEPT_CODE has been added
Nodes CO_RCPT_WTY_TYPE has been added
Nodes CO_COUNS_LARGE_CLASS has been added
Nodes CO_COUNS_MID_CLASS has been added
Nodes CO_COUNS_SMALL_CLASS has been added
Nodes CO_COUNS_STAT has been added
Nodes RP_RCPT_DEPT_TYPE has been added
Nodes RP_RCPT_USER_ID has been added
Nodes RP_REPAIR_SE_ID has been added
Nodes RP_RCPT_TYPE has been added
Nodes RP_RCPT_WTY_TYPE has been added
Nodes RP_RCPT_SYMP_CODE has been added
Nodes RP_RCPT_SUB_SYMP_CODE has been added
Nodes RP_FAIL_CODE_GROUP has been added
Nodes RP_FAIL_LARGE_CLASS has been added
Nodes RP_FAIL_MID_CLASS has been added
Nodes RP_FAIL_REASON_CODE has been added
Nodes RP_FAIL_TYPE has been added
Nodes RP_REPAIR_WTY_TYPE has been added
Nodes RP_TOTAL_FEE_AMT ha

## *거대 Graph delete하려면*

아래는 Cypher query를 program으로 날리는 *implicit transaction* 에서 사용할 수 있는 graph의 nodes와 relationships를 제거하는 방법.  

그러나, 여전히 수 백만 개의 nodes를 갖는 거대 graph에 대해서는 10시간이 넘게 걸린다.  


따라서, 이런 거대 graph에 대해서는, 아래와 같이 graph를 제거한다.  

    1. [neo4j home]/bin/neo4j stop으로 Neo4j server 종료
    2. [neo4j home]/data/ 아래의 databases/ 와 transactions/ directories를 '\rm -rf'로 제거
    3. [neo4j home]/bin/neo4j start로 Neo4j server 재 시작 -> 위 두 directories가 신규 생성된 것을 알 수 있다

In [14]:
cypher_query = \
    '''
    MATCH (n: SERIAL_NO)
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[]

In [54]:
cypher_query = \
    '''
    MATCH (n: CX_TYPE)
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[]

In [46]:
cypher_query = \
    '''
    MATCH (n: CUST_NO)
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[]

In [13]:
cypher_query = \
    '''
    MATCH (n: CX)
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

Query failed: {code: Neo.TransientError.General.OutOfMemoryError} {message: There is not enough memory to perform the current task. Please try increasing 'dbms.memory.heap.max_size' in the neo4j configuration (normally in 'conf/neo4j.conf' or, if you are using Neo4j Desktop, found through the user interface) or if you are running an embedded installation increase the heap by using '-Xmx' command line flag, and then restart the database.}


In [13]:
cypher_query = \
    '''
    MATCH (n)
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

Query failed: {code: Neo.TransientError.General.OutOfMemoryError} {message: There is not enough memory to perform the current task. Please try increasing 'dbms.memory.heap.max_size' in the neo4j configuration (normally in 'conf/neo4j.conf' or, if you are using Neo4j Desktop, found through the user interface) or if you are running an embedded installation increase the heap by using '-Xmx' command line flag, and then restart the database.}


## 5.2. Nodes 간 edges 연결

### 5.2.1. 동일 고객의 CX node와 이전 CX node 연결

In [43]:
def connect_edge(conn, node_type1, node_type2, key1, key2, relation_type):
    cypher_query = \
        "MATCH (n : " f"{node_type1}" "), (m: " f"{node_type2}" ") " \
        "WHERE n." f"{key1}" " = m." f"{key2} " \
        "CREATE (n)-[:" f"{relation_type}" "]->(m)"
    
    conn.query(cypher_query, db = DEFAULT_DATABASE)  

In [44]:
connect_edge(conn, 'CX', 'CX', 'CX_ID_PREV', 'CX_ID', 'REL_CX_ID_PREV')

### 5.2.2. 각 CX node 마다 관련 nodes 연결

Neo4j는 directed relationship만 생성 가능하지만, embedding 시 direction을 무시할 수 있음.

In [45]:
def connect_edge_to_cx(conn, node_name):
    
    node_label_name = "label_"+node_name
    property_name = node_name
    relation_name = "rel_"+node_name

    cypher_query = \
        f'''
        MATCH (label_CX : CX), ({node_label_name}:{node_name})
        WHERE label_CX.{property_name} = {node_label_name}.{property_name}
        CREATE (label_CX)-[:{relation_name}]->({node_label_name})
        '''
    
    conn.query(cypher_query, db = DEFAULT_DATABASE)

In [46]:
# list_node_name에서는 CX_TYPE과 CUST_NO이 제외되었으므로 그건 따로 수행
nm = "CX_TYPE"
connect_edge_to_cx(conn, nm)
print("Relationship rel_{} has been connected".format(nm))

nm = "CUST_NO"
connect_edge_to_cx(conn, nm)
print("Relationship rel_{} has been connected".format(nm))

for nm in list_node_name:
    connect_edge_to_cx(conn, nm)
    print("Relationship rel_{} has been connected".format(nm))

Relationship rel_CX_TYPE has been connected
Relationship rel_CUST_NO has been connected
Relationship rel_CUST_TYPE has been connected
Relationship rel_VIP_CUST_TYPE has been connected
Relationship rel_VIP_GRADE has been connected
Relationship rel_MODEL_CODE has been connected
Relationship rel_PRODUCT_CODE has been connected
Relationship rel_CO_RCPT_CHANNEL has been connected
Relationship rel_CO_RCPT_USER_ID has been connected
Relationship rel_CO_RCPT_DEPT_CODE has been connected
Relationship rel_CO_RCPT_WTY_TYPE has been connected
Relationship rel_CO_COUNS_LARGE_CLASS has been connected
Relationship rel_CO_COUNS_MID_CLASS has been connected
Relationship rel_CO_COUNS_SMALL_CLASS has been connected
Relationship rel_CO_COUNS_STAT has been connected
Relationship rel_RP_RCPT_DEPT_TYPE has been connected
Relationship rel_RP_RCPT_USER_ID has been connected
Relationship rel_RP_REPAIR_SE_ID has been connected
Relationship rel_RP_RCPT_TYPE has been connected
Relationship rel_RP_RCPT_WTY_TYPE has

# 6. Graph projection

Create in-memory graph of  

    (CX)-[:rel_CX_ID_PREV]->(CX)  
    (CX)-[:rel_CX_TYPE]->(CX_TYPE)  
    (CX)-[:rel_CUST_NO]->(CUST_NO)  
    (CX)-[:rel_CUST_TYPE]->(CUST_TYPE)  
    (CX)-[:rel_VIP_CUST_TYPE]->(VIP_CUST_TYPE)  
    (CX)-[:rel_VIP_GRADE]->(VIP_GRADE)  
    (CX)-[:rel_SERIAL_NO]->(SERIAL_NO)  
    (CX)-[:rel_MODEL_CODE]->(MODEL_CODE)  
    (CX)-[:rel_PRODUCT_CODE]->(PRODUCT_CODE)  
    (CX)-[:rel_CO_RCPT_CHANNEL]->(CO_RCPT_CHANNEL)  
    (CX)-[:rel_CO_RCPT_USER_ID]->(CO_RCPT_USER_ID)  
    (CX)-[:rel_CO_RCPT_DEPT_CODE]->(CO_RCPT_DEPT_CODE)  
    (CX)-[:rel_CO_RCPT_WTY_TYPE]->(CO_RCPT_WTY_TYPE)  
    (CX)-[:rel_CO_COUNS_LARGE_CLASS]->(CO_COUNS_LARGE_CLASS)  
    (CX)-[:rel_CO_COUNS_MID_CLASS]->(CO_COUNS_MID_CLASS)  
    (CX)-[:rel_CO_COUNS_SMALL_CLASS]->(CO_COUNS_SMALL_CLASS)  
    (CX)-[:rel_CO_COUNS_STAT]->(CO_COUNS_STAT)  
    (CX)-[:rel_RP_RCPT_DEPT_TYPE]->(RP_RCPT_DEPT_TYPE)  
    (CX)-[:rel_RP_RCPT_USER_ID]->(RP_RCPT_USER_ID)  
    (CX)-[:rel_RP_REPAIR_SE_ID]->(RP_REPAIR_SE_ID)  
    (CX)-[:rel_RP_RCPT_TYPE]->(RP_RCPT_TYPE)  
    (CX)-[:rel_RP_RCPT_WTY_TYPE]->(RP_RCPT_WTY_TYPE)  
    (CX)-[:rel_RP_RCPT_SYMP_CODE]->(RP_RCPT_SYMP_CODE)  
    (CX)-[:rel_RP_RCPT_SUB_SYMP_CODE]->(RP_RCPT_SUB_SYMP_CODE)  
    (CX)-[:rel_RP_FAIL_CODE_GROUP]->(RP_FAIL_CODE_GROUP)  
    (CX)-[:rel_RP_FAIL_LARGE_CLASS]->(RP_FAIL_LARGE_CLASS)  
    (CX)-[:rel_RP_FAIL_MID_CLASS]->(RP_FAIL_MID_CLASS)  
    (CX)-[:rel_RP_FAIL_REASON_CODE]->(RP_FAIL_REASON_CODE)  
    (CX)-[:rel_RP_FAIL_TYPE]->(RP_FAIL_TYPE)  
    (CX)-[:rel_RP_REPAIR_WTY_TYPE]->(RP_REPAIR_WTY_TYPE)  
    (CX)-[:rel_RP_TOTAL_FEE_AMT]->(RP_TOTAL_FEE_AMT)  
    (CX)-[:rel_RP_TOTAL_AMT]->(RP_TOTAL_AMT)  
    (CX)-[:rel_RP_STATUS]->(RP_STATUS)  
    (CX)-[:rel_RP_UNSATIS_ETD_TYPE1]->(RP_UNSATIS_ETD_TYPE1)  
    (CX)-[:rel_RP_UNSATIS_ETD_TYPE2]->(RP_UNSATIS_ETD_TYPE2)  
    (CX)-[:rel_RP_RE_REPAIR_TYPE]->(RP_RE_REPAIR_TYPE)  
    (CX)-[:rel_EX_AGENT_CODE]->(EX_AGENT_CODE)  
    (CX)-[:rel_EX_CAUSE_CODE]->(EX_CAUSE_CODE)  
    (CX)-[:rel_EX_BUY_FACTOR_CODE]->(EX_BUY_FACTOR_CODE)  
    (CX)-[:rel_EX_EXCHANGE_ORDER_TYPE]->(EX_EXCHANGE_ORDER_TYPE)  
    (CX)-[:rel_EX_COMPLETE_YN]->(EX_COMPLETE_YN)  
    (CX)-[:rel_CL_CLAIM_RCPT_CHANNEL]->(CL_CLAIM_RCPT_CHANNEL)  
    (CX)-[:rel_CL_CLAIM_RCPT_TYPE]->(CL_CLAIM_RCPT_TYPE)  
    (CX)-[:rel_CL_RCPT_DEPT_CODE]->(CL_RCPT_DEPT_CODE)  
    (CX)-[:rel_CL_RCPT_USER_ID]->(CL_RCPT_USER_ID)  
    (CX)-[:rel_CL_CLAIM_TYPE]->(CL_CLAIM_TYPE)  
    (CX)-[:rel_CL_CLAIM_LARGE_CLASS]->(CL_CLAIM_LARGE_CLASS)  
    (CX)-[:rel_CL_CLAIM_MID_CLASS]->(CL_CLAIM_MID_CLASS)  
    (CX)-[:rel_CL_CLAIM_SMALL_CLASS]->(CL_CLAIM_SMALL_CLASS)  
    (CX)-[:rel_CL_TGET_SECTION]->(CL_TGET_SECTION)  
    (CX)-[:rel_CL_TGET_DEPT_CODE]->(CL_TGET_DEPT_CODE)  
    (CX)-[:rel_CL_CUST_REQ_TYPE]->(CL_CUST_REQ_TYPE)  
    (CX)-[:rel_CL_PRCS_TYPE]->(CL_PRCS_TYPE)  
    (CX)-[:rel_CL_CLAIM_WTY_TYPE]->(CL_CLAIM_WTY_TYPE)  
    (CX)-[:rel_CL_CLAIM_STATUS]->(CL_CLAIM_STATUS)  

**Syntax**
  
CALL gds.graph.project(  
&nbsp;&nbsp;&nbsp;&nbsp; graphName: String,  
&nbsp;&nbsp;&nbsp;&nbsp; nodeProjection: String or List or Map,  
&nbsp;&nbsp;&nbsp;&nbsp; relationshipProjection: String or List or Map,  
&nbsp;&nbsp;&nbsp;&nbsp; configuration: Map  
) YIELD  
&nbsp;&nbsp;&nbsp;&nbsp; graphName: String,  
&nbsp;&nbsp;&nbsp;&nbsp; nodeProjection: Map,  
&nbsp;&nbsp;&nbsp;&nbsp; nodeCount: Integer,  
&nbsp;&nbsp;&nbsp;&nbsp; relationshipProjection: Map, 
&nbsp;&nbsp;&nbsp;&nbsp; relationshipCount: Integer,  
&nbsp;&nbsp;&nbsp;&nbsp; projectMillis: Integer  

**주의**

Neo4j 4.4.4에 GDS 1.8.6을 사용하면 gds.graph.project()가 동작하지 않으니 주의.  

현재 사용중인 neo4j version은 4.4.4이고 GDS version은 2.1.9임.  

**주의**

추후 12월 CX들을 대상으로 claim 예측 성능 평가를 할텐데, 이 때 공정을 위해 12월 claim CX들은 제외하고 embedding할 것이다.  
왜냐하면, 12월 claim CX들이 포함된 graph로 embedding하면 간접적으로나마 평가 대상인 12월 CX들이 12월 claim 발생을 이용할 수도 있으므로.  

이 때, 12월 claim CX들을 제외하고 embedding하는 방법은, original graph에서 12월 claim CX들을 제외한 sub-graph를 생성해서  
그 sub-graph를 embedding하는 것이다.  

이렇게 sub-graph를 만들려면, original graph의 각 nodes의 properties를 참조할 수 있어야 한다.  

하지만 neo4j community edition은, monopartite graph만 graph projection 가능하다.  
즉, 모든 nodes에 대해서 projection할 property set은 같아야 하는 것.  

따라서, CX가 아닌 nodes들도 참조할 필요가 있는 CX properties를 dummy로 넣어 graph를 구성했고,  
이제 그 graph를 이용해 sub-graph를 projection할 수 있다.  


참고로 성능 평가 방법은 아래와 같다.  

    1. 10, 11월 데이터 만으로 nodes labeling
    2. 12월 데이터들을 합쳐서 original graph 구성
    3. 12월 claim CX nodes를 제외한 sub-graph 구성
    4. Sub-graph로 embedding한 뒤, 12월 CX들의 label 예측 (이미 labeled된 10월, 11월 CX들과의 similarity를 통해)

In [47]:
cypher_query = \
    '''
    // Syntax
    //    CALL gds.graph.project(
    //      graphName: String,
    //      nodeProjection: String or List or Map,
    //      relationshipProjection: String or List or Map,
    //      configuration: Map
    //    ) YIELD
    //      graphName: String,
    //      nodeProjection: Map,
    //      nodeCount: Integer,
    //      relationshipProjection: Map,
    //      relationshipCount: Integer,
    //      projectMillis: Integer
    //
    call gds.graph.project(
        /* Graph Name */
        "SMILE", 
        /* Node projection */
        // <projected-label> : {
        //      label: <neo4j-label>, 
        //      properties: <neo4j-property-key> 
        //  } 
        {
            CX : {label : "CX", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CX_TYPE : {label : "CX_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CUST_NO : {label : "CUST_NO", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CUST_TYPE : {label : "CUST_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            VIP_CUST_TYPE : {label : "VIP_CUST_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            VIP_GRADE : {label : "VIP_GRADE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            SERIAL_NO : {label : "SERIAL_NO", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            MODEL_CODE : {label : "MODEL_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            PRODUCT_CODE : {label : "PRODUCT_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CO_RCPT_CHANNEL : {label : "CO_RCPT_CHANNEL", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CO_RCPT_USER_ID : {label : "CO_RCPT_USER_ID", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CO_RCPT_DEPT_CODE : {label : "CO_RCPT_DEPT_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CO_RCPT_WTY_TYPE : {label : "CO_RCPT_WTY_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CO_COUNS_LARGE_CLASS : {label : "CO_COUNS_LARGE_CLASS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CO_COUNS_MID_CLASS : {label : "CO_COUNS_MID_CLASS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CO_COUNS_SMALL_CLASS : {label : "CO_COUNS_SMALL_CLASS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CO_COUNS_STAT : {label : "CO_COUNS_STAT", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_RCPT_DEPT_TYPE : {label : "RP_RCPT_DEPT_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_RCPT_USER_ID : {label : "RP_RCPT_USER_ID", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_REPAIR_SE_ID : {label : "RP_REPAIR_SE_ID", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_RCPT_TYPE : {label : "RP_RCPT_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_RCPT_WTY_TYPE : {label : "RP_RCPT_WTY_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_RCPT_SYMP_CODE : {label : "RP_RCPT_SYMP_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_RCPT_SUB_SYMP_CODE : {label : "RP_RCPT_SUB_SYMP_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_FAIL_CODE_GROUP : {label : "RP_FAIL_CODE_GROUP", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_FAIL_LARGE_CLASS : {label : "RP_FAIL_LARGE_CLASS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_FAIL_MID_CLASS : {label : "RP_FAIL_MID_CLASS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_FAIL_REASON_CODE : {label : "RP_FAIL_REASON_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_FAIL_TYPE : {label : "RP_FAIL_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_REPAIR_WTY_TYPE : {label : "RP_REPAIR_WTY_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_TOTAL_FEE_AMT : {label : "RP_TOTAL_FEE_AMT", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_TOTAL_AMT : {label : "RP_TOTAL_AMT", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_STATUS : {label : "RP_STATUS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_UNSATIS_ETD_TYPE1 : {label : "RP_UNSATIS_ETD_TYPE1", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_UNSATIS_ETD_TYPE2 : {label : "RP_UNSATIS_ETD_TYPE2", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            RP_RE_REPAIR_TYPE : {label : "RP_RE_REPAIR_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            EX_AGENT_CODE : {label : "EX_AGENT_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            EX_CAUSE_CODE : {label : "EX_CAUSE_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            EX_BUY_FACTOR_CODE : {label : "EX_BUY_FACTOR_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            EX_EXCHANGE_ORDER_TYPE : {label : "EX_EXCHANGE_ORDER_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            EX_COMPLETE_YN : {label : "EX_COMPLETE_YN", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_CLAIM_RCPT_CHANNEL : {label : "CL_CLAIM_RCPT_CHANNEL", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_CLAIM_RCPT_TYPE : {label : "CL_CLAIM_RCPT_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_RCPT_DEPT_CODE : {label : "CL_RCPT_DEPT_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_RCPT_USER_ID : {label : "CL_RCPT_USER_ID", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_CLAIM_TYPE : {label : "CL_CLAIM_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_CLAIM_LARGE_CLASS : {label : "CL_CLAIM_LARGE_CLASS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_CLAIM_MID_CLASS : {label : "CL_CLAIM_MID_CLASS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_CLAIM_SMALL_CLASS : {label : "CL_CLAIM_SMALL_CLASS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_TGET_SECTION : {label : "CL_TGET_SECTION", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_TGET_DEPT_CODE : {label : "CL_TGET_DEPT_CODE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_CUST_REQ_TYPE : {label : "CL_CUST_REQ_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_PRCS_TYPE : {label : "CL_PRCS_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_CLAIM_WTY_TYPE : {label : "CL_CLAIM_WTY_TYPE", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']},
            CL_CLAIM_STATUS : {label : "CL_CLAIM_STATUS", properties : ['CX_TYPE', 'RCPT_DTHM', 'LABEL_CLAIM_NEXT', 'LABEL_CLAIM_FUTURE', 'LABEL_CLAIM', 'LABEL_CLAIM_INF', 'FLAG_TRAIN_INFER']}
        }, 
        /* Relationship projection */
        // <projected-type>: {
        //      type: <neo4j-type>,
        //      orientation: <orientation>,
        //      aggregation: <aggregation-type>,
        //      properties: <neo4j-property-key>
        //  }
        // The orientation can have one of NATURAL, UNDIRECTED, REVERSE
        // The aggregation can have one of NONE, MIN, MAX, SUM, SINGLE, COUNT
        {        
            REL_CX_ID_PREV : {orientation : "NATURAL"},
            rel_CX_TYPE : {orientation : "UNDIRECTED"},
            rel_CUST_NO : {orientation : "UNDIRECTED"},
            rel_CUST_TYPE : {orientation : "UNDIRECTED"},
            rel_VIP_CUST_TYPE : {orientation : "UNDIRECTED"},
            rel_VIP_GRADE : {orientation : "UNDIRECTED"},
            rel_SERIAL_NO : {orientation : "UNDIRECTED"},
            rel_MODEL_CODE : {orientation : "UNDIRECTED"},
            rel_PRODUCT_CODE : {orientation : "UNDIRECTED"},
            rel_CO_RCPT_CHANNEL : {orientation : "UNDIRECTED"},
            rel_CO_RCPT_USER_ID : {orientation : "UNDIRECTED"},
            rel_CO_RCPT_DEPT_CODE : {orientation : "UNDIRECTED"},
            rel_CO_RCPT_WTY_TYPE : {orientation : "UNDIRECTED"},
            rel_CO_COUNS_LARGE_CLASS : {orientation : "UNDIRECTED"},
            rel_CO_COUNS_MID_CLASS : {orientation : "UNDIRECTED"},
            rel_CO_COUNS_SMALL_CLASS : {orientation : "UNDIRECTED"},
            rel_CO_COUNS_STAT : {orientation : "UNDIRECTED"},
            rel_RP_RCPT_DEPT_TYPE : {orientation : "UNDIRECTED"},
            rel_RP_RCPT_USER_ID : {orientation : "UNDIRECTED"},
            rel_RP_REPAIR_SE_ID : {orientation : "UNDIRECTED"},
            rel_RP_RCPT_TYPE : {orientation : "UNDIRECTED"},
            rel_RP_RCPT_WTY_TYPE : {orientation : "UNDIRECTED"},
            rel_RP_RCPT_SYMP_CODE : {orientation : "UNDIRECTED"},
            rel_RP_RCPT_SUB_SYMP_CODE : {orientation : "UNDIRECTED"},
            rel_RP_FAIL_CODE_GROUP : {orientation : "UNDIRECTED"},
            rel_RP_FAIL_LARGE_CLASS : {orientation : "UNDIRECTED"},
            rel_RP_FAIL_MID_CLASS : {orientation : "UNDIRECTED"},
            rel_RP_FAIL_REASON_CODE : {orientation : "UNDIRECTED"},
            rel_RP_FAIL_TYPE : {orientation : "UNDIRECTED"},
            rel_RP_REPAIR_WTY_TYPE : {orientation : "UNDIRECTED"},
            rel_RP_TOTAL_FEE_AMT : {orientation : "UNDIRECTED"},
            rel_RP_TOTAL_AMT : {orientation : "UNDIRECTED"},
            rel_RP_STATUS : {orientation : "UNDIRECTED"},
            rel_RP_UNSATIS_ETD_TYPE1 : {orientation : "UNDIRECTED"},
            rel_RP_UNSATIS_ETD_TYPE2 : {orientation : "UNDIRECTED"},
            rel_RP_RE_REPAIR_TYPE : {orientation : "UNDIRECTED"},
            rel_EX_AGENT_CODE : {orientation : "UNDIRECTED"},
            rel_EX_CAUSE_CODE : {orientation : "UNDIRECTED"},
            rel_EX_BUY_FACTOR_CODE : {orientation : "UNDIRECTED"},
            rel_EX_EXCHANGE_ORDER_TYPE : {orientation : "UNDIRECTED"},
            rel_EX_COMPLETE_YN : {orientation : "UNDIRECTED"},
            rel_CL_CLAIM_RCPT_CHANNEL : {orientation : "UNDIRECTED"},
            rel_CL_CLAIM_RCPT_TYPE : {orientation : "UNDIRECTED"},
            rel_CL_RCPT_DEPT_CODE : {orientation : "UNDIRECTED"},
            rel_CL_RCPT_USER_ID : {orientation : "UNDIRECTED"},
            rel_CL_CLAIM_TYPE : {orientation : "UNDIRECTED"},
            rel_CL_CLAIM_LARGE_CLASS : {orientation : "UNDIRECTED"},
            rel_CL_CLAIM_MID_CLASS : {orientation : "UNDIRECTED"},
            rel_CL_CLAIM_SMALL_CLASS : {orientation : "UNDIRECTED"},
            rel_CL_TGET_SECTION : {orientation : "UNDIRECTED"},
            rel_CL_TGET_DEPT_CODE : {orientation : "UNDIRECTED"},
            rel_CL_CUST_REQ_TYPE : {orientation : "UNDIRECTED"},
            rel_CL_PRCS_TYPE : {orientation : "UNDIRECTED"},
            rel_CL_CLAIM_WTY_TYPE : {orientation : "UNDIRECTED"},
            rel_CL_CLAIM_STATUS : {orientation : "UNDIRECTED"}
        }
        /* Configuration - Global configuration that applies to every node or relationship */
        // { nodeProperties: <common node properties for all nodes> }
        // { relationshipProperties: <common replationship properties for all relationships> }
    ) YIELD
    graphName AS graph,
    nodeProjection AS NodeProjection,
    relationshipProjection AS relProjection,
    nodeCount AS nodes,
    relationshipCount AS rels

    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[<Record graph='SMILE' NodeProjection={'CO_RCPT_CHANNEL': {'label': 'CO_RCPT_CHANNEL', 'properties': {'RCPT_DTHM': {'defaultValue': None, 'property': 'RCPT_DTHM'}, 'LABEL_CLAIM_INF': {'defaultValue': None, 'property': 'LABEL_CLAIM_INF'}, 'LABEL_CLAIM_FUTURE': {'defaultValue': None, 'property': 'LABEL_CLAIM_FUTURE'}, 'FLAG_TRAIN_INFER': {'defaultValue': None, 'property': 'FLAG_TRAIN_INFER'}, 'CX_TYPE': {'defaultValue': None, 'property': 'CX_TYPE'}, 'LABEL_CLAIM': {'defaultValue': None, 'property': 'LABEL_CLAIM'}, 'LABEL_CLAIM_NEXT': {'defaultValue': None, 'property': 'LABEL_CLAIM_NEXT'}}}, 'RP_FAIL_REASON_CODE': {'label': 'RP_FAIL_REASON_CODE', 'properties': {'RCPT_DTHM': {'defaultValue': None, 'property': 'RCPT_DTHM'}, 'LABEL_CLAIM_INF': {'defaultValue': None, 'property': 'LABEL_CLAIM_INF'}, 'LABEL_CLAIM_FUTURE': {'defaultValue': None, 'property': 'LABEL_CLAIM_FUTURE'}, 'FLAG_TRAIN_INFER': {'defaultValue': None, 'property': 'FLAG_TRAIN_INFER'}, 'CX_TYPE': {'defaultValue': None, 'proper

Projection한 graph를 지우려면

In [29]:
cypher_query = \
'''
CALL gds.graph.drop('SMILE') YIELD graphName;
'''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[<Record graphName='SMILE'>]

# 7. Node embedding

지정 가능한 embeddingDimension 값은 128~1024 사이의 2의 승수이다.  
일반적으로 dimension이 클 수록 많은 정보를 담을 수 있어 node 간 분해능이 좋아지지만, 반대급부로 CPU와 memory 소비가 커진다.  
한편, 10^5 즉 10만개 이상의 nodes를 가진 graph에 대해서는 256 이상의 dimension이라야 좋은 분해능이 나온다.

In [48]:
def fastrp_embedding(conn, graph_name, dimension):
    
    # With .write option, all the calculated things are put as a new property (writeProperty)
    #
    # Neo4j supports 3 embedding algorithms: node2vec, FastRP, and GraphSage.
    #     Upon adding a new node, all the graph embedding should be done again for the case of node2vec and FastRP, 
    #     but you don't need to this for GraphSage.
    #     With respect to the node properties, GraphSage supports different properties per node 
    #     (which means it supports multipartite graph). 
    #     But, FastRP supports only a common set of properties for all nodes (i.e., it supports only unipartite graph). 
    #     And, Node2vec does not support node properties at all. 
    # 
    # Syntax.
    # CALL gds.fastRP.write(
    #   graphName: String,
    #   configuration: Map
    # ) YIELD
    #   nodeCount: Integer,
    #   nodePropertiesWritten: Integer,
    #   preProcessingMillis: Integer,
    #   computeMillis: Integer,
    #   writeMillis: Integer,
    #   configuration: Map
    # 
    # Where, important configuration options are:
    # 
    # writeProperty - The node property name to which the embedding result will be written. No default. Must be designated.
    # embeddingDimension - To which dimension, the embedding will be made. No default. Must be designated.
    # randomSeed - No default. Must be designated in integer.
    # 
    # nodeLabels - Target nodes to be embedded. Default = ['*'].
    # relationshipTypes - Target links to be embedded. Default = ['*'].
    # 
    # propertyRatio - Besides topology, how big the impact of node properties should be applied. Default = 0.0.
    # featureProperties - The node properties for propertyRatio. Default = [].
    # normalizationStrength - How to set the importance of high degree nodes. + values increases the importance of them and vice versa. 
    #       Default = 0.0.
    # relationshipWeightProperty - The link property name that designates the relationship weight. Default = Null. 
    #       If all the links of the graph contains the property, one can use this option to tweak the embedding results 
    #       by making them affected by the importance of links.
    # 
    # See https://neo4j.com/docs/graph-data-science/current/machine-learning/node-embeddings/fastrp/ for further information.
    
    cypher_query = \
        "call gds.fastRP.write(  " \
        f"   {graph_name}, " \
        "    { " \
        f"       embeddingDimension : {dimension}, " \
        f"       writeProperty : 'embedding_{dimension}', " \
        "        randomSeed : 32, " \
        "        iterationWeights: [1.0, 1.0, 1.0, 1.0], " \
        "        nodeSelfInfluence: 1.0 " \
        "    } " \
        ") "

    conn.query(cypher_query, db = DEFAULT_DATABASE)

If you want to make embedding for the original graph

In [49]:
fastrp_embedding(conn, '"SMILE"', "256")

Embedding 결과는 각 node의 property의 하나로 저장된다 (gds.fastRP.write()의 writeProperty로 지정된).  

Embedding 완료 후에는 projection된 graph들을 제거한다.

In [24]:
cypher_query = \
'''
CALL gds.graph.drop('SMILE') YIELD graphName;
'''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[<Record graphName='SMILE'>]

# 8. Vector 출력

Embedding을 write mode로 수행함으로써 각 node의 embedding vector는 그 node의 property로 생성되며  
(gds.fastRP.write()의 writeProperty로 지정된 이름의 property에), 이렇게 updated된 graph는 HDD에 저장된다.  

아래 query는 HDD에 저장된 graph의 각 node들을 찾아서 embedding vector 포함 properties를 출력한다.

In [50]:
def format_vectors(conn, node_name, condition):
    
    node_label_name = "label_"+node_name
    property_name = node_name
    relation_name = "rel_"+node_name

    if condition == 'all':
        cypher_query = \
            f'''
            match (n: {node_name}) 
            return n
            '''
    else:
        cypher_query = \
            "MATCH (n: " f"{node_name} " ") " \
            "WHERE " f"{condition} " \
            "RETURN n"
    
    result = conn.query(cypher_query, db = DEFAULT_DATABASE)
    df = pd.DataFrame.from_dict([data['n'] for data in result]).dropna(subset=['embedding_256'])
    #df = pd.DataFrame.from_dict([data['n'] for data in result]).dropna(subset=['embedding_512'])
    #df = pd.DataFrame.from_dict([data['n'] for data in result]).dropna(subset=['embedding_1024'])
    df.to_csv("vectors_"+node_name+".csv", index=False)
    del [[df]]

In [51]:
for nm in list_node_name:
    format_vectors(conn, nm, "all")
    print("Vectors for {} has been output".format(nm))

# list_node_name에서는 CX_TYPE이 제외되었으므로 그건 따로 수행
nm = "CX_TYPE"
format_vectors(conn, nm, "all")
print("Vectors for {} has been output".format(nm))

# list_node_name에서는 CUST_NO이 제외되었으므로 그건 따로 수행
nm = "CUST_NO"
format_vectors(conn, nm, "all")
print("Vectors for {} has been output".format(nm))

# CX도 list_node_name에 없으므로 따로 수행
nm = "CX"
format_vectors(conn, nm, "all")
print("Vectors for {} has been output".format(nm))
#os.system("mv -f vectors_CX.csv vectors_CX_2210.csv")

Vectors for CUST_TYPE has been output
Vectors for VIP_CUST_TYPE has been output
Vectors for VIP_GRADE has been output
Vectors for MODEL_CODE has been output
Vectors for PRODUCT_CODE has been output
Vectors for CO_RCPT_CHANNEL has been output
Vectors for CO_RCPT_USER_ID has been output
Vectors for CO_RCPT_DEPT_CODE has been output
Vectors for CO_RCPT_WTY_TYPE has been output
Vectors for CO_COUNS_LARGE_CLASS has been output
Vectors for CO_COUNS_MID_CLASS has been output
Vectors for CO_COUNS_SMALL_CLASS has been output
Vectors for CO_COUNS_STAT has been output
Vectors for RP_RCPT_DEPT_TYPE has been output
Vectors for RP_RCPT_USER_ID has been output
Vectors for RP_REPAIR_SE_ID has been output
Vectors for RP_RCPT_TYPE has been output
Vectors for RP_RCPT_WTY_TYPE has been output
Vectors for RP_RCPT_SYMP_CODE has been output
Vectors for RP_RCPT_SUB_SYMP_CODE has been output
Vectors for RP_FAIL_CODE_GROUP has been output
Vectors for RP_FAIL_LARGE_CLASS has been output
Vectors for RP_FAIL_MID_C

새로운 graph를 만들려면 HDD에 저장된 graph를 삭제해야 한다

In [14]:
cypher_query = \
    '''
    MATCH (n: SERIAL_NO)
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[]

In [14]:
cypher_query = \
    '''
    MATCH (n: CUST_NO)
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[]

In [14]:
cypher_query = \
    '''
    MATCH (n: CX)
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[]

In [14]:
cypher_query = \
    '''
    MATCH (n)
    CALL {
        WITH n
        DETACH DELETE n
    } IN TRANSACTIONS
    '''

conn.query(cypher_query, db = DEFAULT_DATABASE)

[]

# 9. Vector를 train set과 inference set으로 분리

이는, 일관성 유지를 위해 train set과 inference set을 한 번에 embedding한 경우에 대한 것

In [52]:
# Input data 경로설정
curr_dir = os.getcwd()
print(curr_dir)

os.chdir("/home/sookyoung.kim/ssd/projects/GraphDB/customer_index/complaint_index")
#os.system('ls')
train_infer_data_path = os.getcwd()
print(train_infer_data_path)
os.chdir(curr_dir)

## Input file 명 설정
train_infer_data = os.path.join(train_infer_data_path, "vectors_CUST_NO.csv")
print(train_infer_data)

/ssd_data/sookyoung_ssd/projects/GraphDB/customer_index/complaint_index
/ssd_data/sookyoung_ssd/projects/GraphDB/customer_index/complaint_index
/ssd_data/sookyoung_ssd/projects/GraphDB/customer_index/complaint_index/vectors_CUST_NO.csv


In [53]:
df_vector_train_infer = pd.read_csv(train_infer_data)

# Train set
cond_tmp = (df_vector_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 0) | (df_vector_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 2)
df_vector_train = df_vector_train_infer.loc[cond_tmp, :]

df_vector_train.to_csv('vectors_CUST_NO-train.csv', index=False)

# Inference set
cond_tmp = (df_vector_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 1) | (df_vector_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 2)
df_vector_infer = df_vector_train_infer.loc[cond_tmp, :]

# Inference set은 LABEL_CLAIM_INF를 LABEL_CLAIM에 복사
df_vector_infer['LABEL_CLAIM'] = df_vector_infer['LABEL_CLAIM_INF']

df_vector_infer.to_csv('vectors_CUST_NO-infer.csv', index=False)

/home/sookyoung.kim/.local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
